In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer
import core.utils as utils
import numpy as np
import matplotlib.pyplot as plt

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

In [ ]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FullRecoTransformer(data_config, name="Transformer")

In [ ]:
y_train["regression"] = np.concatenate(
    [
        utils.lorentz_vector_from_neutrino_momenta_array(y_train["regression"]),
        utils.lorentz_vector_from_neutrino_momenta_array(y_train["regression"])
        + X_train["lepton_truth"],
    ],
    axis=1,
)

In [ ]:
W_4vect = utils.lorentz_vector_from_neutrino_momenta_array(X_train["regression"]) + X_train["lepton_truth"]
W_beta = -W_4vect[... ,:3] / W_4vect[..., 3:4]

In [ ]:
W_mass = utils.compute_mass_from_lorentz_vector_array(W_4vect)
plt.hist(W_mass.flatten(), bins=100)

In [ ]:
rf_loss = utils.RestframeLoss()
neutrino_boosted = np.array(rf_loss.boost(utils.lorentz_vector_from_neutrino_momenta_array(X_train["regression"]), W_beta))
lepton_boosted = np.array(rf_loss.boost(X_train["lepton_truth"], W_beta))

In [ ]:
boosted_W_4vect = neutrino_boosted + lepton_boosted

In [ ]:
plt.hist(boosted_W_4vect[..., 3].flatten(), bins=100)

In [ ]:
angle = (utils.angle_vectors(neutrino_boosted[:,0 ,:3],lepton_boosted[:,0, :3])).flatten() / np.pi
import matplotlib.pyplot as plt
plt.hist(angle, bins=50)

In [ ]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
)

In [ ]:
Transformer.adapt_normalization_layers(X_train)
Transformer.add_reco_mass_deviation_loss()
Transformer.compile_model(
    loss={
        "assignment": utils.AssignmentLoss(),
        "normalized_regression": utils.RestframeLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
    metrics={
        "assignment": [utils.AssignmentAccuracy(name="accuracy")],
        #"normalized_regression": [utils.RegressionDeviation(name="deviation")],
    },
    loss_weights={"assignment": 1.0, "normalized_regression": 10.0, "reco_mass_deviation": 5.0},
    add_physics_informed_loss=True
)

In [ ]:
X_train, y_train, sample_weights = Transformer.prepare_training_data(
    X_train,
    y_train,
    copy_data=True,
    sample_weights=utils.compute_sample_weights(X_train, y_train),
)

In [ ]:
Transformer.train_model(
    epochs=20,
    X=X_train,
    y=y_train,
    sample_weight=sample_weights,
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
        keras.callbacks.TerminateOnNaN(),
    ],
    validation_split=0.1,
)

In [ ]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")

In [ ]:
wrapped_model = Transformer.export_to_onnx(MODEL_DIR + "odd_model.onnx")